In [5]:
from __future__ import print_function
#import tensorflow.keras as keras # расскоментируйте эту строку, чтобы начать обучение
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.utils import to_categorical

import os

# установка параметров нейросети
batch_size = 32
num_classes = 10
epochs = 10
data_augmentation = True
num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_trained_model.h5'

# разделение тренировочной и тестовой выборки
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[1:], 'тренировочные примеры')
print(x_test.shape[1:], 'тестовые примеры')

x_train shape: (50000, 32, 32, 3)
(32, 32, 3) тренировочные примеры
(32, 32, 3) тестовые примеры


In [0]:
# преобразование матрицы чисел 0-9 в бинарную матрицу чисел 0-1
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255


In [7]:

# конфигурирование слоев нейросети
model = Sequential()

# слои нейросети отвественные за свертку и max-pooling
model.add(Conv2D(256, (5, 5), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(3, 3), strides=2))

model.add(Conv2D(384, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(384, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(256, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(3, 3), strides=2))

# полносвязные слои нейронной сети
model.add(Flatten())
model.add(Dense(4096))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(4096))
model.add(Activation('relu'))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

# компиляция модели
model.compile(loss='categorical_crossentropy',
              optimizer='SGD',
              metrics=['accuracy'])

print('Не используется data augmentation')
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_test, y_test),
          shuffle=True)


# проверка работы обученной модели
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])


Не используется data augmentation
Epoch 1/10
1563/1563 [==============================] - 48s 31ms/step - loss: 1.9760 - accuracy: 0.2753 - val_loss: 1.7322 - val_accuracy: 0.3721
Epoch 2/10
1563/1563 [==============================] - 47s 30ms/step - loss: 1.5586 - accuracy: 0.4338 - val_loss: 1.5377 - val_accuracy: 0.4568
Epoch 3/10
1563/1563 [==============================] - 48s 30ms/step - loss: 1.3475 - accuracy: 0.5151 - val_loss: 1.3808 - val_accuracy: 0.5093
Epoch 4/10
1563/1563 [==============================] - 48s 30ms/step - loss: 1.1948 - accuracy: 0.5740 - val_loss: 1.1044 - val_accuracy: 0.6058
Epoch 5/10
1563/1563 [==============================] - 48s 30ms/step - loss: 1.0608 - accuracy: 0.6254 - val_loss: 0.9916 - val_accuracy: 0.6530
Epoch 6/10
1563/1563 [==============================] - 47s 30ms/step - loss: 0.9455 - accuracy: 0.6686 - val_loss: 1.0394 - val_accuracy: 0.6455
Epoch 7/10
1563/1563 [==============================] - 48s 30ms/step - loss: 0.8480 - acc

In [8]:
#оценим рост сложности при увеличении ширины сети (больше ядер)
# конфигурирование слоев нейросети
model = Sequential()

# слои нейросети отвественные за свертку и max-pooling
model.add(Conv2D(300, (5, 5), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(3, 3), strides=2))

model.add(Conv2D(400, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(400, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(300, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(3, 3), strides=2))

# полносвязные слои нейронной сети
model.add(Flatten())
model.add(Dense(4096))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(4096))
model.add(Activation('relu'))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

# компиляция модели
model.compile(loss='categorical_crossentropy',
              optimizer='SGD',
              metrics=['accuracy'])

print('Не используется data augmentation')
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_test, y_test),
          shuffle=True)


# проверка работы обученной модели
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])


Не используется data augmentation
Epoch 1/10
1563/1563 [==============================] - 58s 37ms/step - loss: 1.9733 - accuracy: 0.2755 - val_loss: 1.7725 - val_accuracy: 0.3604
Epoch 2/10
1563/1563 [==============================] - 57s 36ms/step - loss: 1.5513 - accuracy: 0.4364 - val_loss: 1.4014 - val_accuracy: 0.4902
Epoch 3/10
1563/1563 [==============================] - 57s 36ms/step - loss: 1.3495 - accuracy: 0.5159 - val_loss: 1.3832 - val_accuracy: 0.5185
Epoch 4/10
1563/1563 [==============================] - 57s 37ms/step - loss: 1.1921 - accuracy: 0.5765 - val_loss: 1.3277 - val_accuracy: 0.5352
Epoch 5/10
1563/1563 [==============================] - 57s 37ms/step - loss: 1.0647 - accuracy: 0.6234 - val_loss: 1.0443 - val_accuracy: 0.6286
Epoch 6/10
1563/1563 [==============================] - 57s 36ms/step - loss: 0.9483 - accuracy: 0.6660 - val_loss: 0.9754 - val_accuracy: 0.6555
Epoch 7/10
1563/1563 [==============================] - 57s 36ms/step - loss: 0.8479 - acc

In [10]:
#оценим рост сложности при увеличении глубины сети (больше слоев)
#
# конфигурирование слоев нейросети
model = Sequential()

# слои нейросети отвественные за свертку и max-pooling
model.add(Conv2D(256, (5, 5), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(3, 3), strides=2))

model.add(Conv2D(384, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(384, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(256, (3, 3), padding='same'))
model.add(Activation('relu'))
#добавим еще один слой!
model.add(Conv2D(256, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(3, 3), strides=2))

#добавим еще один слой!
model.add(Conv2D(256, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(3, 3), strides=2))

# полносвязные слои нейронной сети
model.add(Flatten())
model.add(Dense(4096))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(4096))
model.add(Activation('relu'))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

# компиляция модели
model.compile(loss='categorical_crossentropy',
              optimizer='SGD',
              metrics=['accuracy'])

print('Не используется data augmentation')
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_test, y_test),
          shuffle=True)


# проверка работы обученной модели
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])


Не используется data augmentation
Epoch 1/10
1563/1563 [==============================] - 48s 31ms/step - loss: 2.1211 - accuracy: 0.2057 - val_loss: 1.8988 - val_accuracy: 0.2908
Epoch 2/10
1563/1563 [==============================] - 47s 30ms/step - loss: 1.7363 - accuracy: 0.3635 - val_loss: 1.5981 - val_accuracy: 0.4113
Epoch 3/10
1563/1563 [==============================] - 48s 30ms/step - loss: 1.5063 - accuracy: 0.4484 - val_loss: 1.3912 - val_accuracy: 0.4877
Epoch 4/10
1563/1563 [==============================] - 47s 30ms/step - loss: 1.3466 - accuracy: 0.5127 - val_loss: 1.3481 - val_accuracy: 0.5335
Epoch 5/10
1563/1563 [==============================] - 47s 30ms/step - loss: 1.2053 - accuracy: 0.5707 - val_loss: 1.1501 - val_accuracy: 0.5841
Epoch 6/10
1563/1563 [==============================] - 47s 30ms/step - loss: 1.0786 - accuracy: 0.6190 - val_loss: 1.0394 - val_accuracy: 0.6306
Epoch 7/10
1563/1563 [==============================] - 47s 30ms/step - loss: 0.9653 - acc

Вывод: точность получившейся модели - 0.74. Если крутить рулетку, то точность такого случайного классификатора получится = 0.1. Получившийся классификатор в 7 раз лучше случайного. Увеличение кол-ва ядер и слоев не принесло увеличения точности.  